# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up Turi Create

In [2]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [44]:
sales = turicreate.SFrame('home_data.sframe/')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [19]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [45]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = turicreate.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                    validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training Max Error | Training Root-Mean-Square Error |

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.019992     | 4146407.590959     | 258679.804259                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [46]:
example_weight_summary = example_model.coefficients
print (example_weight_summary)

+-------------+-------+---------------------+--------------------+
|     name    | index |        value        |       stderr       |
+-------------+-------+---------------------+--------------------+
| (intercept) |  None |  87910.06920678669  | 7873.338136755249  |
| sqft_living |  None |  315.4034411181024  | 3.4557003229375387 |
|   bedrooms  |  None | -65080.215441625194 | 2717.4568521267206 |
|  bathrooms  |  None |  6944.020876313873  | 3923.1149281297144 |
+-------------+-------+---------------------+--------------------+
[4 rows x 4 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [47]:
example_predictions = example_model.predict(train_data)
print (example_predictions[0]) # should be 271789.505878

271789.50427758583


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [48]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions

    # Then compute the residuals/errors

    # Then square and add them up
    predictions = model.predict(data)
    
    # Then compute the residuals/errors
    errors = outcome - predictions
    
    # Then square and add them up
    square_errors = errors * errors
    RSS = square_errors.sum()
    

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [49]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print (rss_example_train) # should be 2.7376153833e+14

273761534465419.88


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [50]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [51]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [52]:
# create the remaining 3 features in both TEST and TRAIN data
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']


test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

print ("bedrooms_squared: " + str(test_data['bedrooms_squared'].mean()))
print ("bed_bath_rooms: " + str(test_data['bed_bath_rooms'].mean()) )
print ("log_sqft_living: " + str(test_data['log_sqft_living'].mean())) 
print ("lat_plus_long: " + str(test_data['lat_plus_long'].mean()))


bedrooms_squared: 12.44667770158429
bed_bath_rooms: 7.503901631591395
log_sqft_living: 7.55027467964594
lat_plus_long: -74.6533335540317


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [53]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [54]:
# Learn the three models: (don't forget to set validation_set = None)

model_1 = turicreate.linear_regression.create(train_data, target = 'price', features = model_1_features, 
                                                    validation_set = None)
model_2 = turicreate.linear_regression.create(train_data, target = 'price', features = model_2_features, 
                                                    validation_set = None)
model_3 = turicreate.linear_regression.create(train_data, target = 'price', features = model_3_features, 
                                                    validation_set = None)


# features_1 = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
# model_1 = turicreate.linear_regression.create(train_data, target = 'price', features = features_1, validation_set = None)



Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

| 1         | 2        | 0.015556     | 4074792.266563     | 236378.065704                   |

| 2         | 3        | 0.038159     | 4074792.266501     | 236378.065703                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

ToolkitError: Input data does not contain the following columns: ['bed_bath_rooms']

In [43]:
model_1 = turicreate.linear_regression.create(train_data, target = 'price', features = model_1_features, validation_set = None)
# model_2 = turicreate.linear_regression.create(train_data, target = 'price', features = model_2_features, validation_set = None)
model_3 = turicreate.linear_regression.create(train_data, target = 'price', features = model_3_features, validation_set = None)


Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

| 1         | 2        | 0.014369     | 4074792.266563     | 236378.065704                   |

| 2         | 3        | 0.034631     | 4074792.266501     | 236378.065703                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

ToolkitError: Input data does not contain the following columns: ['lat_plus_long', 'bed_bath_rooms', 'log_sqft_living']

In [34]:
# Examine/extract each model's coefficients:
model_1_summary = model_1.coefficients
print (model_1_summary)
model_2_summary = model_2.coefficients
print (model_2_summary)
model_3_summary = model_3.coefficients
print (model_3_summary)



+-------------+-------+--------------------+--------------------+
|     name    | index |       value        |       stderr       |
+-------------+-------+--------------------+--------------------+
| (intercept) |  None | -56141852.46435269 | 1649928.3580122495 |
| sqft_living |  None | 310.2635067230146  | 3.1888206232738048 |
|   bedrooms  |  None | -59577.60521214531 | 2487.2745584621066 |
|  bathrooms  |  None | 13812.406875960849 | 3593.5344057244292 |
|     lat     |  None | 629863.1199344355  | 13120.688670221312 |
|     long    |  None | -214800.9523004764 | 13283.820389035558 |
+-------------+-------+--------------------+--------------------+
[6 rows x 4 columns]



NameError: name 'model_2' is not defined

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [ ]:
# Compute the RSS on TRAINING data for each of the three models and record the values:


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [ ]:
# Compute the RSS on TESTING data for each of the three models and record the values:


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.